# How to Migrate minto.v0 Experiment to minto.v1

This notebook is a guide to help you migrate your minto.v0 experiment to minto.v1.

In [1]:
import minto
import numpy as np
import jijmodeling as jm

In [2]:
tsp = minto.problems.tsp.QuadTSP()
tsp_problem = tsp.problem()
tsp_data = tsp.data(n=8)

## minto.v0 experiment

You can use the `v0.x.x` version of the minto in `v1.x.x` with `minto.v1`.
In the following, we create a simple experiment using `minto.v0` and then migrate it to `minto.v1`.

In [3]:
import jijmodeling_transpiler as jmt

compiled_instance = jmt.core.compile_model(tsp_problem, tsp_data)
qubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_instance)
qubo, _ = qubo_builder.get_qubo_dict()

In [4]:
import minto.v0
import openjij as oj

sampler = oj.SASampler()
sweeps_list = np.arange(100, 1000, 100)

exp_v0 = minto.v0.Experiment("v0_exp")
for sweeps in sweeps_list:
    with exp_v0.run():
        response = sampler.sample_qubo(qubo, num_reads=10, num_sweeps=sweeps)
        sampleset = jmt.core.pubo.openjij_decode.decode_from_openjij(
            response,
            qubo_builder,
            compiled_instance
        )
        sampleset = jm.experimental.from_old_sampleset(sampleset)
        exp_v0.log_result("result", sampleset)
        exp_v0.log_parameter("sweeps", sweeps)


In [5]:
exp_v0.table(enable_sampleset_expansion=False)

,experiment_name,run_id,sweeps,result
0,v0_exp,0,100,"(Sample(run_id=""b62a278d-91eb-4413-aa4d-1f6df4..."
1,v0_exp,1,200,"(Sample(run_id=""80153cc8-a3ec-42de-9199-13f121..."
2,v0_exp,2,300,"(Sample(run_id=""fa4d1964-8a02-40d8-acd4-a01eb9..."
3,v0_exp,3,400,"(Sample(run_id=""cc854ff9-fb96-424c-ae5b-10ed4e..."
4,v0_exp,4,500,"(Sample(run_id=""421e6577-b98f-4a21-a0a3-4ca75b..."
5,v0_exp,5,600,"(Sample(run_id=""7a9526ee-46cb-4791-b994-cb376c..."
6,v0_exp,6,700,"(Sample(run_id=""37557e74-9854-490d-a8c0-92062c..."
7,v0_exp,7,800,"(Sample(run_id=""64a6df02-c64e-46ec-9c05-4dc2a4..."
8,v0_exp,8,900,"(Sample(run_id=""ace33c00-8ac4-43b0-81b9-4a3d4a..."


## Migrate from minto.v0 to minto.v1

To migrate from `minto.v0` to `minto.v1`, you need to use `migrate_to_v1_from_v0` function.

In [6]:
from minto.migrator.v0tov1 import migrate_to_v1_from_v0

exp_v1 = migrate_to_v1_from_v0(exp_v0)

In [7]:
exp_v1.get_run_table()

sampleset_result                                                   \
            num_samples  obj_mean   obj_std   obj_min   obj_max feasible   
run_id                                                                     
0                    10  3.522054  0.390917  2.883244  4.207748       10   
1                    10  3.421715  0.495696  2.869669  4.380822       10   
2                    10  3.105068  0.494227  2.661808  4.098958       10   
3                    10  3.198603  0.354913  2.831967  3.925243       10   
4                    10  3.037024  0.241666  2.700304  3.529449       10   
5                    10  3.153340  0.350452  2.661808  3.869771       10   
6                    10  3.276451  0.363259  2.709548  3.735491       10   
7                    10  3.223950  0.388657  2.700304  3.665643       10   
8                    10  3.274044  0.326622  2.987036  3.910863       10   

                                  parameter metadata               
       feasible_unrelaxed    name    sweeps   run_id elapsed_time  
run_id                                                             
0                      10  result       100        0     0.001561  
1                      10  result       200        1     0.001403  
2                      10  result       300        2     0.001363  
3                      10  result       400        3     0.001273  
4                      10  result       500        4     0.001263  
5                      10  result       600        5     0.001405  
6                      10  result       700        6     0.001281  
7                      10  result       800        7     0.001315  
8                      10  result       900        8     0.001294

Then you can save the migrated experiment using `.save()` or `.save_as_ommx_archive` function.

In [8]:
# exp_v1.save()  # save the experiment at disk
# exp_v1.save_as_ommx_archive("v1_exp.ommx")  # save the experiment as an ommx archive